<a href="https://colab.research.google.com/github/chewzzz1014/fyp/blob/master/ner/src/train_ner_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train NER Models

In [1]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!mkdir spacy_ner_data

## Spacy NER

In [3]:
# load json and convert into spacy format

import json
import random
from sklearn.model_selection import train_test_split
import spacy
import numpy as np
np.float_ = np.float64
from spacy.tokens import DocBin

# load JSON data from Drive
with open('/content/drive/MyDrive/FYP/Implementation/Resume Dataset/1100_resumes_annotated.json', "r") as f:
    data = json.load(f)

# remove overlapped entities (one word has >1 entitiy)
def remove_overlapping_entities(entities):
    """Remove overlapping entities from the list."""
    # sort entities by start position
    entities = sorted(entities, key=lambda x: x[0])
    non_overlapping = []
    last_end = -1
    for start, end, label in entities:
        # only add to list if there's no overlap with the previous entity
        if start >= last_end:
            non_overlapping.append((start, end, label))
            last_end = end
    return non_overlapping

# convert JSON data to Spacy's DocBin format
def convert_to_spacy_format(data):
    # load a blank Spacy model
    nlp = spacy.blank("en")
    # container for our docs
    doc_bin = DocBin()

    for item in data:
        # full document text
        text = item['data']['Text']
        entities = []

        for annotation in item['annotations'][0]['result']:
            start = annotation['value']['start']
            end = annotation['value']['end']
            label = annotation['value']['labels'][0]
            entities.append((start, end, label))

        # remove overlapping entities
        entities = remove_overlapping_entities(entities)
        # create a Spacy doc and add entities to it
        doc = nlp.make_doc(text)
        spans = [doc.char_span(start, end, label=label) for start, end, label in entities]
        # filter out None spans if Spacy can't align the character indices with tokens
        spans = [span for span in spans if span is not None]
        # assign entities to the doc
        doc.ents = spans
        doc_bin.add(doc)

    return doc_bin

# split data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# convert train and test sets to Spacy format
train_doc_bin = convert_to_spacy_format(train_data)
test_doc_bin = convert_to_spacy_format(test_data)

# save the train and test data to .spacy files in current runtime
train_doc_bin.to_disk("spacy_ner_data/train_data.spacy")
test_doc_bin.to_disk("spacy_ner_data/test_data.spacy")

In [4]:
# check the distribution of entitiy labels

import spacy
from spacy.lang.en import English
from spacy.tokens import DocBin

# count number of label entities
def count_entity_labels(file_path):
    # load data from file_path and create DocBin
    doc_bin = DocBin().from_disk(file_path)
    label_counts = {}
    for doc in doc_bin.get_docs(English().vocab):
        # count occurence of label
        for ent in doc.ents:
            label = ent.label_
            label_counts[label] = label_counts.get(label, 0) + 1
    return label_counts

# calculate and print label distribution in train and test data
# sorted from largest to smallest
train_label_counts = count_entity_labels("spacy_ner_data/train_data.spacy")
sorted_train_label_counts = sorted(train_label_counts.items(), key=lambda x: x[1], reverse=True)
print("Train Data Entity Label Distribution:")
for label, count in sorted_train_label_counts:
    print(f"{label}: {count}")

test_label_counts = count_entity_labels("spacy_ner_data/test_data.spacy")
sorted_test_label_counts = sorted(test_label_counts.items(), key=lambda x: x[1], reverse=True)
print("\nTest Data Entity Label Distribution:")
for label, count in sorted_test_label_counts:
    print(f"{label}: {count}")

Train Data Entity Label Distribution:
SKILL: 15539
JOB: 4275
LOC: 2963
WORK PER: 2795
COMPANY: 2606
UNI: 1249
DEG: 1091
NAME: 883
STUDY PER: 852
PHONE: 816
EMAIL: 775

Test Data Entity Label Distribution:
SKILL: 3812
JOB: 1063
LOC: 778
WORK PER: 687
COMPANY: 640
UNI: 325
DEG: 282
NAME: 225
STUDY PER: 212
PHONE: 205
EMAIL: 189


In [5]:
# create base_config.cfg and paste the config generated from spacy widget
# need to update train and test file path
!touch base_config.cfg

In [6]:
# install transformer
!pip install git+https://github.com/explosion/spacy-transformers

  Cloning https://github.com/explosion/spacy-transformers to /tmp/pip-req-build-qhcpto2d
  Running command git clone --filter=blob:none --quiet https://github.com/explosion/spacy-transformers /tmp/pip-req-build-qhcpto2d
  Resolved https://github.com/explosion/spacy-transformers to commit 40ee09d9b2b2b18f77fc3715329ce080691b6af9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.6 MB/s eta 0:00:00
  Created wheel for spacy-transformers: filename=spacy_transformers-1.3.6-cp310-cp310-linux_x86_64.whl size=680951 sha256=4c8547b7749ed7b7a6c0fd2d295726663d74a61769846f79ba24953bceb34aff
  Stored in directory: /tmp/pip-

In [7]:
!pip install -U spacy
!pip install "numpy<2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 101.1 MB/s eta 0:00:00
  Attempting uninstall: blis
    Found existing installation: blis 0.7.11
    Uninstalling blis-0.7.11:
      Successfully uninstalled blis-0.7.11
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.5
    Uninstalling thinc-8.2.5:
      Successfully uninstalled thinc-8.2.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.5
    Uninstalling spacy-3.7.5:
      Successfully uninstalled spacy-3.7.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.7.1 requires spacy<3.8.0,>=3.7.2, but you have spacy 3.8.3 which is incompatible.


In [8]:
# generate config.cfg from base_config.cfg
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
# vector used for spacy ner
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 2.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [10]:
# debugging and profiling data.
!python -m spacy debug data config.cfg


============================ Data file validation ============================
tokenizer_config.json: 100% 25.0/25.0 [00:00<00:00, 150kB/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
config.json: 100% 481/481 [00:00<00:00, 2.70MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 4.17MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 2.13MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 3.13MB/s]
model.safetensors: 100% 499M/499M [00:02<00:00, 232MB/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
✔ Pipeline can be i

In [11]:
# debugging and profiling configs and implementations.
!python -m spacy debug config config.cfg


============================= Config validation =============================

===================== Config validation for [initialize] =====================

====================== Config validation for [training] ======================
✔ Config is valid


In [12]:
# train model using hyperparameters set in config.cfg
# save trained model in spacy_output/ dir

# using cpu
# !python -m spacy train config.cfg --output ./spacy_output

# using gpu
!python -m spacy train config.cfg --gpu-id 0 --output ./spacy_output

# save output dir into drive
!cp -r ./spacy_output /content/drive/MyDrive/FYP/Implementation/

✔ Created output directory: spacy_output
ℹ Saving to output directory: spacy_output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ---

In [13]:
# evaluate trained model performance
# store output and visualization into result/ dir

# use cpu
# !python -m spacy evaluate spacy_output/model-best spacy_ner_data/test_data.spacy -dp spacy_output

# use gpu
!python -m spacy evaluate spacy_output/model-best spacy_ner_data/test_data.spacy --gpu-id 0 -dp spacy_output

ℹ Using GPU: 0
/usr/local/lib/python3.10/dist-packages/spacy_transformers/layers/hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self._model.load_state_dict(to

In [14]:
# make prediction

import spacy_transformers
import spacy
import numpy as np
np.float_ = np.float64
import string
from spacy import displacy

resume_text = '''
John Doe lives at 1234 Elm Street in Los Angeles, CA 90001. He can be reached at +1 (555) 123-4567 or via email at john.doe@example.com. John is a results-driven software engineer with over 5 years of experience in web development and cloud infrastructure, with strong knowledge of JavaScript, Python, and cloud technologies like AWS and Azure. Currently, he works as a Software Engineer at Google LLC in San Francisco, CA, where he has been employed since August 2019. In this role, he has developed scalable web applications using JavaScript, Node.js, and React, deployed and maintained cloud infrastructure on AWS, reducing downtime by 20%, and led a team of 4 engineers to enhance backend performance by 30%. Previously, he worked as a Junior Developer at Tech Innovators Inc. in Austin, TX, from July 2017 to July 2019, where he created RESTful APIs using Python and Flask, collaborated with front-end developers to build and deploy user-facing applications, and wrote unit and integration tests, improving code coverage by 15%.

John holds a Master of Science in Computer Science from the University of California, Berkeley, with a graduation date of May 2017, and a Bachelor of Science in Information Technology from the University of Texas at Austin, graduated in May 2015. His skillset includes proficiency in programming languages like Python, JavaScript, and Java; frameworks such as React, Flask, and Django; cloud platforms including AWS, Google Cloud, and Azure; as well as other tools like Git, Docker, Kubernetes, and SQL. He is certified as an AWS Certified Solutions Architect – Associate, earned in 2020, and as a Google Professional Cloud Architect, earned in 2021'
'''

# convert text into small letter then remove punctuation
resume_text = resume_text.lower()
resume_text = resume_text.translate(str.maketrans('', '', string.punctuation))

# load trained model
nlp = spacy.load("/content/drive/MyDrive/FYP/Implementation/spacy_output/model-best")

# create a Spacy doc and add text to it
doc = nlp(resume_text.lower())

# print predicted entities in text
# for ent in doc.ents:
#     print(f"{ent.text}: {ent.label_}")

# visualize predicted entities using displacy
colors = {
    "NAME": "lightblue",
    "LOC": "yellow",
    "PHONE": "pink",
    "EMAIL": "lightgreen",
    "JOB": "orange",
    "SKILL": "aqua",
    "COMPANY": "violet",
    "WORK PER": "salmon",
    "DEG": "lightcoral",
    "UNI": "lightgrey",
    "STUDY PER": "peachpuff",
}
options = {"ents": list(colors.keys()), "colors": colors}
displacy.render(doc, style="ent", jupyter=True, options=options)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.8.3 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
/usr/local/lib/python3.10/dist-packages/spacy_transformers/layers/hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pi

In [15]:
# make prediction

import spacy
import string
from spacy import displacy

resume_text = '''
Zi Qing Chew
chewziqing@gmail.com | 016-2892475 | Kuala Lumpur, Malaysia | linkedin.com/in/ziqingchew | github.com/chewzzz1014
EDUCATION

Universiti Putra Malaysia					                                                   Oct 2021 - Current
Bachelor in Computer Science with Honours
Expected to graduate in July 2025. CGPA: 3.99

WORK EXPERIENCE

Ant International 									          	July 2024 – Oct 2024
Java Engineer Intern							                               Kuala Lumpur, Malaysia
Collaborated in developing an audit logging feature for Ant Group’s internal Foreign Exchange (FX) trade strategy system that records changes made by business users to trade strategies.
Conducted comprehensive system analysis and project planning, delivering presentations to project stakeholders and QA teams prior to the development phase.
Utilised Ant Group’s internal frameworks, middleware, and tools to implement the audit logging feature.
Skills: Java, Spring, Sofaboot, Ant Group internal middlewares (ZDAL, DRM, Ant Scheduler, Msg Broker)
Howuku  									          	             Feb 2023 – Sep 2023
Software Developer Intern							                    Kuala Lumpur, Malaysia
Developed and optimized A/B testing features, including code editor and previewer for CSS and JavaScript modifications for experiment variations.
Expanded A/B testing targeting rule by incorporating website visitor's OS, device, and browser rules.
Automated experiment-stopping criteria and email notifications based on user-defined experiment termination conditions.
Collaborated with cross-functional teams to debug, troubleshoot, and enhance Howuku platform features based on user feedback and performance data.
Skills: JavaScript, Bootstrap, Vue.js, Express.js, MySQL

PROJECTS

Personal Portfolio Website (chewzzz1014.github.io/portfolio-website)
Designed, developed and deployed personalised portfolio website featuring skills, selected projects, and downloadable resume.
Skills: JavaScript, React.js, CSS, Bootstrap
Depression Level Detection Chatbot (https://github.com/chewzzz1014/health-ease-project)
Developed machine learning application that evaluates a message's depression level and provided tailored mental health advice and information based on the depression severity.
Skills: Python, pandas, scikit-learn, Keras, FastAPI, Gradio
Clothing Store Website (https://github.com/chewzzz1014/CSC3402-MVC-Project)
Worked in team to build a CRUD Spring Boot application with attractive interfaces, data persistence, authentication and authorisation.
Developed the backend of the application that involves querying the database, building REST endpoints and implementing Thymeleaf in HTML for dynamic contents.
Skills: Spring Boot, Spring MVC, Thymeleaf, Hibernate, Bootstrap

SKILLS
Programming Languages: Java, Python, HTML, CSS, JavaScript, MySQL, OracleSQL
Frameworks and Libraries: Spring, Spring Boot, TypeScript, Node.js, Express.js, React.js, Vue.js, Bootstrap, Tailwind CSS
Tools: Git, Github, Jira, Tableau, Excel, Jupyter Notebook, Google Colab, VSCode, IntelliJ
'''

# convert text into small letter then remove punctuation
# resume_text = resume_text.lower()
resume_text = resume_text.translate(str.maketrans('', '', string.punctuation))

# create a Spacy doc and add text to it
doc = nlp(resume_text)

# load trained model
nlp = spacy.load("/content/drive/MyDrive/FYP/Implementation/spacy_output/model-best")

colors = {
    "NAME": "lightblue",
    "LOC": "yellow",
    "PHONE": "pink",
    "EMAIL": "lightgreen",
    "JOB": "orange",
    "SKILL": "aqua",
    "COMPANY": "violet",
    "WORK PER": "salmon",
    "DEG": "lightcoral",
    "UNI": "lightgrey",
    "STUDY PER": "peachpuff",
}
options = {"ents": list(colors.keys()), "colors": colors}
displacy.render(doc, style="ent", jupyter=True, options=options)

/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  # NB: Previously this was torch.cuda.amp.autocast, passing a boolean
/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.8.3 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/spacy_transformers/layers/hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbit

## Flair NER

In [ ]:
# install flair library
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 22.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 8.2 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=16a403eda69d9e25a27603d4168e35fb6fa64b91b428b6c0aaf83a375cb08650
  Stored in directory: /root/.cac

In [ ]:
# convert json into flair data

import json
import random
from typing import List, Dict, Tuple
import spacy
from collections import defaultdict

class NERConverter:
    def __init__(self):
        # load pretrained model from Spacy library
        # to create Spacy Doc object
        self.nlp = spacy.load("en_core_web_sm")

    # get BIOES label based on location of word
    def get_bioes_label(self, token_index: int, entity_length: int, current_position: int, label: str) -> str:
        """
        Convert to BIOES format
        - S-: Single token entity
        - B-: Beginning of multi-token entity
        - I-: Inside of multi-token entity
        - E-: End of multi-token entity
        - O: Outside
        """
        if entity_length == 1:
            return f'S-{label}'
        if current_position == 0:
            return f'B-{label}'
        if current_position == entity_length - 1:
            return f'E-{label}'
        return f'I-{label}'

    # convert Label Studio's exported annotations in json format intto BIOES format
    def convert_to_bioes_format(self, json_data: List[dict]) -> List[List[Tuple[str, str]]]:
        """Convert JSON annotations to BIOES format."""
        all_sentences = []

        # process all annotation in json file
        for item in json_data:
            text = item['data']['Text']
            doc = self.nlp(text)

            # initialize character-level labels
            char_labels = ['O'] * len(text)

            # first pass: identify entity boundaries and lengths
            entity_spans = []
            if item['annotations'] and len(item['annotations']) > 0:
                for ann in item['annotations'][0]['result']:
                    if 'value' in ann:
                        start = ann['value']['start']
                        end = ann['value']['end']
                        label = ann['value']['labels'][0]
                        entity_spans.append((start, end, label))

            # sort spans by start position
            entity_spans.sort(key=lambda x: x[0])

            # second pass: apply BIOES labels
            for start, end, label in entity_spans:
                # get tokens that are part of this entity
                entity_text = text[start:end]
                entity_doc = self.nlp(entity_text)
                entity_length = len([token for token in entity_doc if not token.is_space])

                # set labels for the entire span
                current_token_idx = 0
                for i in range(start, end):
                    if i == start or text[i-1].isspace():
                        char_labels[i] = self.get_bioes_label(i, entity_length, current_token_idx, label)
                        current_token_idx += 1
                    else:
                        char_labels[i] = char_labels[i-1]

            # convert to token-level labels
            current_sentence = []
            for sent in doc.sents:
                for token in sent:
                    # get the most common label for the token's characters
                    token_chars_labels = char_labels[token.idx:token.idx + len(token.text)]
                    label_counts = defaultdict(int)
                    for char_label in token_chars_labels:
                        label_counts[char_label] += 1

                    token_label = max(label_counts.items(), key=lambda x: x[1])[0]
                    current_sentence.append((token.text, token_label))

                if current_sentence:
                    all_sentences.append(current_sentence)
                    current_sentence = []

        return all_sentences

    # write data in BIOES format into txt file
    def write_flair_file(self, sentences: List[List[Tuple[str, str]]], filename: str):
        """Write sentences in BIOES format to file."""
        with open(filename, 'w', encoding='utf-8') as f:
            for sentence in sentences:
                for token, label in sentence:
                    f.write(f'{token} {label}\n')
                f.write('\n')

    # convert json data into BIOES data
    # split BIOES data into train and test
    def convert_and_split(self, json_data: List[dict], train_file: str, test_file: str, test_ratio: float = 0.2):
        """Convert JSON to BIOES format and split into train/test sets."""
        all_sentences = self.convert_to_bioes_format(json_data)

        # shuffle and split based on test_ratio
        random.shuffle(all_sentences)
        split_idx = int(len(all_sentences) * (1 - test_ratio))

        # use list slicing to split
        train_sentences = all_sentences[:split_idx]
        test_sentences = all_sentences[split_idx:]

        # write to txt files
        self.write_flair_file(train_sentences, train_file)
        self.write_flair_file(test_sentences, test_file)

        return len(train_sentences), len(test_sentences)


# load JSON data
with open('/content/drive/MyDrive/FYP/Implementation/Resume Dataset/1100_resumes_annotated.json', 'r', encoding='utf-8') as f:
  json_data = json.load(f)
# with open('/content/1100_resumes_annotated.json', 'r', encoding='utf-8') as f:
#   json_data = json.load(f)

# load self-defined convert class
converter = NERConverter()

# convert json data into BIOES data and split into train and test
train_count, test_count = converter.convert_and_split(
    json_data,
    train_file='flair_train.txt',
    test_file='flair_test.txt',
    test_ratio=0.2
)
print(f'Created {train_count} training sentences and {test_count} test sentences')

Created 1585 training sentences and 397 test sentences


In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# define columns for CoNLL (0: word, 1: label)
columns = {0: 'text', 1: 'ner'}

# set data folder and train and test path
data_folder = './'
train_file = 'flair_train.txt'
test_file = 'flair_test.txt'

# load the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file=train_file,
                              test_file=test_file,
                              dev_file=None)

2024-12-12 08:49:56,886 Reading data from .
2024-12-12 08:49:56,887 Train: flair_train.txt
2024-12-12 08:49:56,888 Dev: None
2024-12-12 08:49:56,889 Test: flair_test.txt
2024-12-12 08:50:03,698 No dev split found. Using 10% (i.e. 158 samples) of the train split as dev data


In [ ]:
# generate a dictionary of unique labels from the NER corpus.
# this dictionary maps each named entity label in the dataset to an integer ID.
tag_dictionary = corpus.make_label_dictionary(label_type='ner')
print("Labels:", tag_dictionary.get_items())

2024-12-12 08:50:07,914 Computing label dictionary. Progress:


0it [00:00, ?it/s]
1427it [00:00, 15331.24it/s]

2024-12-12 08:50:08,055 Dictionary created for label 'ner' with 11 values: SKILL (seen 13598 times), JOB (seen 4002 times), LOC (seen 2659 times), WORK (seen 2596 times), COMPANY (seen 2355 times), UNI (seen 1175 times), DEG (seen 993 times), NAME (seen 788 times), STUDY (seen 778 times), PHONE (seen 752 times), EMAIL (seen 693 times)
Labels: ['SKILL', 'JOB', 'LOC', 'WORK', 'COMPANY', 'UNI', 'DEG', 'NAME', 'STUDY', 'PHONE', 'EMAIL']


In [ ]:
from collections import Counter

# count frequency of each entity label
def count_labels(file_path):
    with open(file_path, 'r') as file:
        labels = [line.split()[-1] for line in file if line.strip()]
    return Counter(labels)

# number of
print("Train label distribution:", count_labels('flair_train.txt'))
print("Test label distribution:", count_labels('flair_test.txt'))

Train label distribution: Counter({'O': 376460, 'S-SKILL': 7652, 'B-SKILL': 7475, 'E-SKILL': 7475, 'PER': 6842, 'E-JOB': 3911, 'B-JOB': 3857, 'I-JOB': 2108, 'B-COMPANY': 2064, 'E-COMPANY': 2063, 'S-LOC': 1937, 'I-DEG': 1852, 'I-SKILL': 1473, 'I-COMPANY': 1419, 'E-UNI': 1250, 'B-UNI': 1247, 'I-UNI': 1148, 'E-DEG': 1071, 'B-DEG': 1070, 'E-LOC': 964, 'B-LOC': 962, 'E-NAME': 870, 'B-NAME': 868, 'S-EMAIL': 754, 'E-PHONE': 674, 'B-PHONE': 665, 'S-COMPANY': 529, 'S-JOB': 510, 'I-PHONE': 472, 'S-PHONE': 139, 'I-NAME': 33, 'I-LOC': 32, 'S-UNI': 31, 'S-DEG': 21, 'B-EMAIL': 3, 'E-EMAIL': 3, 'S-NAME': 1})
Test label distribution: Counter({'O': 87769, 'S-SKILL': 2029, 'B-SKILL': 1883, 'E-SKILL': 1878, 'PER': 1612, 'E-JOB': 920, 'B-JOB': 914, 'I-JOB': 508, 'E-COMPANY': 480, 'B-COMPANY': 477, 'S-LOC': 456, 'I-DEG': 434, 'I-SKILL': 374, 'I-COMPANY': 349, 'E-UNI': 283, 'B-UNI': 282, 'B-DEG': 262, 'E-DEG': 261, 'I-UNI': 246, 'E-LOC': 241, 'B-LOC': 240, 'B-NAME': 235, 'E-NAME': 235, 'S-EMAIL': 191, 'B-PH

In [ ]:
# create NER tagger
from flair.embeddings import WordEmbeddings, StackedEmbeddings, TransformerWordEmbeddings, FlairEmbeddings
from flair.models import SequenceTagger
import torch.nn as nn

# 1. using LSTM-CRF on top of frozen embeddings
# combine flair and glove embeddings
embeddings = StackedEmbeddings([
    WordEmbeddings('glove'),  # GloVe word embeddings
    FlairEmbeddings('news-forward'),  # Slightly heavier version of the forward Flair embeddings
    FlairEmbeddings('news-backward')  # Slightly heavier version of the backward Flair embeddings
])

# 2. Configure tagger with memory and performance optimizations
tagger = SequenceTagger(
    hidden_size=128,  # Increased hidden size for more capacity
    embeddings=embeddings,
    tag_dictionary=tag_dictionary,
    tag_type='ner',
    use_crf=True,
    tag_format='BIOES',
    dropout=0.2,  # Reduced dropout for better capacity retention
    rnn_layers=2,  # Increased layers to enhance representation learning
)

2024-12-12 08:50:23,733 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmptcelcvdt


100%|██████████| 153M/153M [00:09<00:00, 16.4MB/s]

2024-12-12 08:50:34,014 copying /tmp/tmptcelcvdt to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2024-12-12 08:50:34,205 removing temp file /tmp/tmptcelcvdt
2024-12-12 08:50:34,751 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmptg1z30xv


100%|██████████| 20.5M/20.5M [00:02<00:00, 9.23MB/s]

...

2024-12-12 08:50:52,381 copying /tmp/tmp0s0e0krx to cache at /root/.flair/embeddings/news-forward-0.4.1.pt
2024-12-12 08:50:52,462 removing temp file /tmp/tmp0s0e0krx


2024-12-12 08:50:54,551 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-backward-0.4.1.pt not found in cache, downloading to /tmp/tmp1ki_xkkt


100%|██████████| 69.7M/69.7M [00:06<00:00, 11.3MB/s]

2024-12-12 08:51:01,539 copying /tmp/tmp1ki_xkkt to cache at /root/.flair/embeddings/news-backward-0.4.1.pt
2024-12-12 08:51:01,617 removing temp file /tmp/tmp1ki_xkkt


2024-12-12 08:51:01,912 SequenceTagger predicts: Dictionary with 45 tags: O, S-SKILL, B-SKILL, E-SKILL, I-SKILL, S-JOB, B-JOB, E-JOB, I-JOB, S-LOC, B-LOC, E-LOC, I-LOC, S-WORK, B-WORK, E-WORK, I-WORK, S-COMPANY, B-COMPANY, E-COMPANY, I-COMPANY, S-UNI, B-UNI, E-UNI, I-UNI, S-DEG, B-DEG, E-DEG, I-DEG, S-NAME, B-NAME, E-NAME, I-NAME, S-STUDY, B-STUDY, E-STUDY, I-STUDY, S-PHONE, B-PHONE, E-PHONE, I-PHONE, S-EMAIL, B-EMAIL, E-EMAIL, I-EMAIL


In [ ]:
# train flair ner model

from flair.trainers import ModelTrainer
from flair.training_utils import EvaluationMetric
import torch

# define ModelTrained based on tagger and corpus
trainer = ModelTrainer(tagger, corpus)

# train Flair NER Model
trainer.train(
    base_path='flair_output/',
    learning_rate=0.05,  # Lower learning rate for more stable training
    mini_batch_size=16,  # Increased batch size (if memory permits)
    max_epochs=50,  # More epochs to allow better model convergence
    patience=5,  # Increased patience for early stopping
    train_with_dev=True,
    save_final_model=True,
    use_amp=True,  # Mixed precision training for faster training
)

# save trained model to drive
!cp -r ./flair_output /content/drive/MyDrive/FYP/Implementation/

2024-12-04 12:06:51,350 ----------------------------------------------------------------------------------------------------
2024-12-04 12:06:51,352 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
      )
    )
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4196, out_features=4196, bias=True)
  (rnn): LSTM(4196, 128, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (

/usr/local/lib/python3.10/dist-packages/flair/trainers/trainer.py:499: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2024-12-04 12:07:32,760 epoch 1 - iter 7/73 - loss 4.19854115 - time (sec): 41.38 - samples/sec: 769.52 - lr: 0.050000 - momentum: 0.000000
2024-12-04 12:08:18,910 epoch 1 - iter 14/73 - loss 4.20430475 - time (sec): 87.52 - samples/sec: 742.59 - lr: 0.050000 - momentum: 0.000000
2024-12-04 12:09:19,855 epoch 1 - iter 21/73 - loss 3.58259553 - time (sec): 148.47 - samples/sec: 653.92 - lr: 0.050000 - momentum: 0.000000
2024-12-04 12:10:06,312 epoch 1 - iter 28/73 - loss 2.94078037 - time (sec): 194.93 - samples/sec: 654.54 - lr: 0.050000 - momentum: 0.000000
2024-12-04 12:10:51,804 epoch 1 - iter 35/73 - loss 2.53528460 - time (sec): 240.42 - samples/sec: 655.64 - lr: 0.050000 - momentum: 0.000000
2024-12-04 12:11:31,874 epoch 1 - iter 42/73 - loss 2.27277629 - time (sec): 280.49 - samples/sec: 666.33 - lr: 0.050000 - momentum: 0.000000
2024-12-04 12:12:15,286 epoch 1 - iter 49/73 - loss 2.06623933 - time (sec): 323.90 - samples/sec: 669.77 - lr: 0.050000 - momentum: 0.000000
2024-12-0

  0%|          | 0/5 [00:17<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.38 GiB. GPU 0 has a total capacity of 14.75 GiB of which 1.19 GiB is free. Process 3542 has 13.55 GiB memory in use. Of the allocated memory 11.10 GiB is allocated by PyTorch, and 2.31 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# evaluate model

from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

# load the trained model
tagger = SequenceTagger.load('/content/drive/MyDrive/FYP/Implementation/flair_output/final-model.pt')
# tagger = SequenceTagger.load('/content/final-model.pt')

# evaluate the model on the test set
result = tagger.evaluate(corpus.test, gold_label_type='ner', mini_batch_size=32)

# print the results
# print("Evaluation Loss:", eval_loss)
print(result.detailed_results)  # print the precision, recall, and F1-score per entity type

2024-12-12 08:51:38,310 SequenceTagger predicts: Dictionary with 47 tags: O, S-SKILL, B-SKILL, E-SKILL, I-SKILL, S-JOB, B-JOB, E-JOB, I-JOB, S-LOC, B-LOC, E-LOC, I-LOC, S-WORK, B-WORK, E-WORK, I-WORK, S-COMPANY, B-COMPANY, E-COMPANY, I-COMPANY, S-UNI, B-UNI, E-UNI, I-UNI, S-DEG, B-DEG, E-DEG, I-DEG, S-NAME, B-NAME, E-NAME, I-NAME, S-STUDY, B-STUDY, E-STUDY, I-STUDY, S-PHONE, B-PHONE, E-PHONE, I-PHONE, S-EMAIL, B-EMAIL, E-EMAIL, I-EMAIL, <START>, <STOP>


100%|██████████| 13/13 [02:12<00:00, 10.18s/it]


Results:
- F-score (micro) 0.648
- F-score (macro) 0.789
- Accuracy 0.4826

By class:
              precision    recall  f1-score   support

       SKILL     0.4602    0.5290    0.4922      3913
         JOB     0.5969    0.7389    0.6604      1038
         LOC     0.7701    0.9082    0.8334       697
        WORK     0.8596    0.9259    0.8915       688
     COMPANY     0.6742    0.7855    0.7256       606
         UNI     0.6715    0.7938    0.7276       291
         DEG     0.7381    0.8127    0.7736       267
        NAME     0.9425    0.8950    0.9181       238
       PHONE     0.9518    0.9559    0.9538       227
       STUDY     0.6933    0.8418    0.7604       196
       EMAIL     0.9078    0.9791    0.9421       191

   micro avg     0.6063    0.6960    0.6480      8352
   macro avg     0.7514    0.8333    0.7890      8352
weighted avg     0.6105    0.6960    0.6499      8352



In [ ]:
# make prediction

from flair.models import SequenceTagger
from flair.data import Sentence
import spacy, string
from spacy import displacy

# load trained Flair NER model
# tagger = SequenceTagger.load('/content/drive/MyDrive/FYP/Implementation/flair_output/best-model.pt')
# tagger = SequenceTagger.load('/content/final-model.pt')

resume_text = '''
John Doe lives at 1234 Elm Street in Los Angeles, CA 90001. He can be reached at +1 (555) 123-4567 or via email at john.doe@example.com. John is a results-driven software engineer with over 5 years of experience in web development and cloud infrastructure, with strong knowledge of JavaScript, Python, and cloud technologies like AWS and Azure. Currently, he works as a Software Engineer at Google LLC in San Francisco, CA, where he has been employed since August 2019. In this role, he has developed scalable web applications using JavaScript, Node.js, and React, deployed and maintained cloud infrastructure on AWS, reducing downtime by 20%, and led a team of 4 engineers to enhance backend performance by 30%. Previously, he worked as a Junior Developer at Tech Innovators Inc. in Austin, TX, from July 2017 to July 2019, where he created RESTful APIs using Python and Flask, collaborated with front-end developers to build and deploy user-facing applications, and wrote unit and integration tests, improving code coverage by 15%.

John holds a Master of Science in Computer Science from the University of California, Berkeley, with a graduation date of May 2017, and a Bachelor of Science in Information Technology from the University of Texas at Austin, graduated in May 2015. His skillset includes proficiency in programming languages like Python, JavaScript, and Java; frameworks such as React, Flask, and Django; cloud platforms including AWS, Google Cloud, and Azure; as well as other tools like Git, Docker, Kubernetes, and SQL. He is certified as an AWS Certified Solutions Architect – Associate, earned in 2020, and as a Google Professional Cloud Architect, earned in 2021'
'''

# make into all small letter and remove punctuations
resume_text = resume_text.lower()
resume_text = resume_text.translate(str.maketrans('', '', string.punctuation))

# step 1: predict entities using Flair trained model
sentence = Sentence(resume_text)
tagger.predict(sentence)

# step 2: convert Flair predictions to spaCy doc format
# initialize a blank spaCy NLP pipeline
nlp = spacy.blank("en")
doc = nlp(resume_text)

# extract entities from Flair prediction and convert to spaCy format
ents = []
for entity in sentence.get_spans('ner'):
    start, end = entity.start_position, entity.end_position
    label = entity.tag
    span = doc.char_span(start, end, label=label)
    if span is not None:
        ents.append(span)

# set the entities in the spaCy doc
doc.ents = ents

# step 3: visualization of prediction using displacy
displacy.render(doc, style="ent", jupyter=True)


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
resume_text_1 = '''
Zi Qing Chew
chewziqing@gmail.com | 016-2892475 | Kuala Lumpur, Malaysia | linkedin.com/in/ziqingchew | github.com/chewzzz1014
EDUCATION

Universiti Putra Malaysia					                                                   Oct 2021 - Current
Bachelor in Computer Science with Honours
Expected to graduate in July 2025. CGPA: 3.99

WORK EXPERIENCE

Ant International 									          	July 2024 – Oct 2024
Java Engineer Intern							                               Kuala Lumpur, Malaysia
Collaborated in developing an audit logging feature for Ant Group’s internal Foreign Exchange (FX) trade strategy system that records changes made by business users to trade strategies.
Conducted comprehensive system analysis and project planning, delivering presentations to project stakeholders and QA teams prior to the development phase.
Utilised Ant Group’s internal frameworks, middleware, and tools to implement the audit logging feature.
Skills: Java, Spring, Sofaboot, Ant Group internal middlewares (ZDAL, DRM, Ant Scheduler, Msg Broker)
Howuku  									          	             Feb 2023 – Sep 2023
Software Developer Intern							                    Kuala Lumpur, Malaysia
Developed and optimized A/B testing features, including code editor and previewer for CSS and JavaScript modifications for experiment variations.
Expanded A/B testing targeting rule by incorporating website visitor's OS, device, and browser rules.
Automated experiment-stopping criteria and email notifications based on user-defined experiment termination conditions.
Collaborated with cross-functional teams to debug, troubleshoot, and enhance Howuku platform features based on user feedback and performance data.
Skills: JavaScript, Bootstrap, Vue.js, Express.js, MySQL

PROJECTS

Personal Portfolio Website (chewzzz1014.github.io/portfolio-website)
Designed, developed and deployed personalised portfolio website featuring skills, selected projects, and downloadable resume.
Skills: JavaScript, React.js, CSS, Bootstrap
Depression Level Detection Chatbot (https://github.com/chewzzz1014/health-ease-project)
Developed machine learning application that evaluates a message's depression level and provided tailored mental health advice and information based on the depression severity.
Skills: Python, pandas, scikit-learn, Keras, FastAPI, Gradio
Clothing Store Website (https://github.com/chewzzz1014/CSC3402-MVC-Project)
Worked in team to build a CRUD Spring Boot application with attractive interfaces, data persistence, authentication and authorisation.
Developed the backend of the application that involves querying the database, building REST endpoints and implementing Thymeleaf in HTML for dynamic contents.
Skills: Spring Boot, Spring MVC, Thymeleaf, Hibernate, Bootstrap

SKILLS
Programming Languages: Java, Python, HTML, CSS, JavaScript, MySQL, OracleSQL
Frameworks and Libraries: Spring, Spring Boot, TypeScript, Node.js, Express.js, React.js, Vue.js, Bootstrap, Tailwind CSS
Tools: Git, Github, Jira, Tableau, Excel, Jupyter Notebook, Google Colab, VSCode, IntelliJ
'''

In [ ]:
# make prediction
from flair.models import SequenceTagger
from flair.data import Sentence
import spacy, string
from spacy import displacy


# load trained Flair NER model
# tagger = SequenceTagger.load('/content/drive/MyDrive/FYP/Implementation/flair_output/best-model.pt')
# tagger = SequenceTagger.load('/content/final-model.pt')

resume_text_1 = resume_text_1.lower()
resume_text_1 = resume_text_1.translate(str.maketrans('', '', string.punctuation))

# step 1: predict entities using Flair trained model
sentence = Sentence(resume_text_1)
tagger.predict(sentence)


# step 2: convert Flair predictions to spaCy doc format
# initialize a blank spaCy NLP pipeline
nlp = spacy.blank("en")
doc = nlp(resume_text_1)


# extract entities from Flair prediction and convert to spaCy format
ents = []
for entity in sentence.get_spans('ner'):
    start, end = entity.start_position, entity.end_position
    label = entity.tag
    span = doc.char_span(start, end, label=label)
    if span is not None:
        ents.append(span)



# set the entities in the spaCy doc
doc.ents = ents



# step 3: visualization of prediction using displacy
displacy.render(doc, style="ent", jupyter=True)